In [6]:
import ee
import ipyleaflet
import mercantile
import os

BUCKET = os.environ.get('EE_BUCKET')

ee.Initialize()

In [5]:
#!earthengine authenticate --quiet

In [4]:
#!earthengine authenticate --authorization-code

In [7]:
# utility funcs
def GetTileLayerUrl(eeImage):
  map_id = ee.Image(eeImage).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

def TileLayer(eeImage):
  return ipyleaflet.TileLayer(url=GetTileLayerUrl(eeImage))

def ShowLayer(eeImage):
  Map = ipyleaflet.Map(center=(48.2, 16.3), zoom=2, layout={'height':'200px'})
  Map.add_layer(TileLayer(eeImage))
  return Map

In [8]:
# video tiles
def createExportVideoTileTask(eeImageCollection, tile, bucket, prefix, **args):
  geom = mercantile.feature(tile)['geometry']['coordinates']
  pfx = '{}/{}/{}/'.format(prefix, tile.z, tile.x)
  fname = '{}'.format(tile.y)
  desc = (pfx+fname).replace('/','_')
  task = ee.batch.Export.video.toCloudStorage(eeImageCollection, 
    description=desc,
    bucket=bucket,
    fileNamePrefix=pfx+fname,
    dimensions=[256,256],
    region=geom,
    **args)
  return task

def exportVideoTiles(eeImageCollection, zoom_levels, bucket, prefix, **args):
  tasks = []
  for tile in mercantile.tiles(-180, -85, 180, 85, zoom_levels, truncate=False):
    task = createExportVideoTileTask(eeImageCollection, tile, bucket, prefix, **args)
    task.start()
    tasks.append(task)
  return tasks

In [9]:
# get image collection
ic = ee.ImageCollection('users/resourcewatchlandcover/LandUseHarmonization/Historical/Baseline/States')
ic = ic.sort('year')

#print(ic.aggregate_array('year'))
ic.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'urban',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [1440, 720],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0.0, -180.0, 0.0, -0.25, 90.0]},
  {'id': 'c3ann',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [1440, 720],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0.0, -180.0, 0.0, -0.25, 90.0]},
  {'id': 'c4ann',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [1440, 720],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0.0, -180.0, 0.0, -0.25, 90.0]},
  {'id': 'c3per',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [1440, 720],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0.0, -180.0, 0.0, -0.25, 90.0]},
  {'id': 'c4per',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [1440, 720],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0.0, -180.0, 0.0, -0.25, 90.0]},
  {'id':

In [11]:
# Visualization
# Bands are % of cell in land cover class, compute simple linear average of colors

i1 = ee.Image.constant(1)
gamma = 1
color_constants = {
  'c0': i1.visualize(palette=['gray']), 
  'c1': i1.visualize(palette=['yellow']),
  'c2': i1.visualize(palette=['lightgreen']),
  'c3': i1.visualize(palette=['yellow']),
  'c4': i1.visualize(palette=['lightgreen']),
  'c5': i1.visualize(palette=['yellow']),
  'c6': i1.visualize(palette=['lightgreen']),
  'c7': i1.visualize(palette=['lightgreen']),
  'c8': i1.visualize(palette=['darkgreen']),
  'c9': i1.visualize(palette=['darkgreen']),
  'c10': i1.visualize(palette=['tan']),
  'c11': i1.visualize(palette=['tan']),
}

def viz(i): 
  return i.expression(
    'b(0)*c0 + b(1)*c1 + b(2)*c2 +' +
    'b(3)*c3 + b(4)*c4 + b(5)*c5 +' +
    'b(6)*c6 + b(7)*c7 + b(8)*c8 +' +
    'b(9)*c9 + b(10)*c10 + b(11)*c11'
    , color_constants).toUint8()

In [12]:
y1 = 1900
y2 = 2000

frames = ic.filterMetadata('year','not_less_than', y1).filterMetadata('year','not_greater_than', y2).map(viz)
im = ee.Image(frames.first())
ShowLayer(im)

Map(center=[48.2, 16.3], layers=(TileLayer(options=[u'opacity', u'attribution', u'max_zoom', u'detect_retina', u'min_zoom', u'tile_size']), TileLayer(options=[u'opacity', u'attribution', u'max_zoom', u'detect_retina', u'min_zoom', u'tile_size'], url=u'https://earthengine.googleapis.com/map/4254fd262e6abb86c1ff3b2ec9bc81cd/{z}/{x}/{y}?token=0f9208741c1d48980e1e0876f742b808')), layout=Layout(height=u'200px'), options=[u'keyboard_pan_offset', u'tap', u'attribution_control', u'max_zoom', u'min_zoom', u'bounce_at_zoom_limits', u'keyboard', u'scroll_wheel_zoom', u'dragging', u'inertia_max_speed', u'close_popup_on_click', u'zoom_control', u'box_zoom', u'double_click_zoom', u'tap_tolerance', u'zoom_start', u'keyboard_zoom_offset', u'inertia_deceleration', u'inertia', u'center', u'zoom', u'world_copy_jump', u'zoom_animation_threshold', u'touch_zoom'], zoom=2)

In [ ]:
layer_name = 'LUH_historical'
zoom_levels = [0, 1]
bucket = 'ee-export-744hn9bz'

tasks = exportVideoTiles(frames, zoom_levels, bucket, layer_name)

In [13]:
ee.batch.Task.list()

[<Task EXPORT_VIDEO: LUH_historical_1_1_1 (COMPLETED)>,
 <Task EXPORT_VIDEO: LUH_historical_1_1_0 (COMPLETED)>,
 <Task EXPORT_VIDEO: LUH_historical_1_0_1 (COMPLETED)>,
 <Task EXPORT_VIDEO: LUH_historical_1_0_0 (COMPLETED)>,
 <Task EXPORT_VIDEO: LUH_historical_0_0_0 (COMPLETED)>,
 <Task EXPORT_VIDEO: LUH_historical_0_0_0 (COMPLETED)>,
 <Task EXPORT_VIDEO: LUH_historical_0_0_0 (COMPLETED)>,
 <Task EXPORT_VIDEO: LUH_historical_2_3_3 (COMPLETED)>,
 <Task EXPORT_VIDEO: LUH_historical_2_3_2 (COMPLETED)>,
 <Task EXPORT_VIDEO: LUH_historical_2_3_1 (COMPLETED)>,
 <Task EXPORT_VIDEO: LUH_historical_2_3_0 (COMPLETED)>,
 <Task EXPORT_VIDEO: LUH_historical_2_2_3 (COMPLETED)>,
 <Task EXPORT_VIDEO: LUH_historical_2_2_2 (COMPLETED)>,
 <Task EXPORT_VIDEO: LUH_historical_2_2_1 (COMPLETED)>,
 <Task EXPORT_VIDEO: LUH_historical_2_2_0 (COMPLETED)>,
 <Task EXPORT_VIDEO: LUH_historical_2_1_3 (COMPLETED)>,
 <Task EXPORT_VIDEO: LUH_historical_2_1_2 (COMPLETED)>,
 <Task EXPORT_VIDEO: LUH_historical_2_1_1 (COMPL